# 这个版本错误挺多！

有诸多问题...

In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim 

# 超参数
gamma = 0.99  # 折扣因子
lambda_ = 0.95  # GAE平滑参数
epsilon = 0.2  # PPO剪切参数
beta = 0.01  # KL惩罚系数
value_loss_rate = 0.5  # 值损失权重
epochs = 10  # 每批次训练轮数

# actor_model是策略模型，即要训练的大模型
# critic_model是状态价值模型，还是动作价值价值模型？这个模型是来自哪里？怎么设计这个模型的网络结构
# 优化器
actor_optimizer = optim.Adam(actor_model.parameters(), lr=1e-5)
critic_optimizer = optim.Adam(critic_model.parameters(), lr=1e-5)

# 工具函数
def gather_log_probs(logits, actions):
    """从logits中提取实际动作的对数概率"""
    return torch.log_softmax(logits, dim=-1).gather(dim=-1, index=actions.unsqueeze(-1))

# 这个优势计算，
def compute_advantages(values, rewards, gamma=0.99, lambda_=0.95):
    """计算GAE优势"""
    advantages = torch.zeros_like(rewards)
    gae = 0
    for t in reversed(range(rewards.size(1))):
        """
        (t < rewards.size(1)-1) 当达到序列最后一个元素是，输出false, 即0,未来奖励无 -> 这里反向操作，意味着，达到序列初始位置，前面无奖励
        (t < rewards.size(1)-1) 当未达到序列最后一个元素时，输出为True, 即１,有未来奖励　-> 这里反向操作，意味着，未达到序列初始位置，前面有奖励
        
        疑问：这里每个状态下，动作的即使奖励是怎么计算得来的？
        """
        delta = rewards[:, t] + gamma * values[:, t+1] * (t < rewards.size(1)-1) - values[:, t]
        gae = delta + gamma * lambda_ * gae
        advantages[:, t] = gae
    return advantages


# 训练循环
for batch_prompt in  prompt_dataset:
    # 同一批次的提示词，要固定相同的长度对吗？最终模型输出也是相同长度，对吗？
    batch_size, prompt_len = batch_prompt.size() # [batch_size, prompt_len]
    
    # 生成响应
    # 输出生成序列的同时，输出token的概率
    batch_response, old_log_probs = actor_model.generate(batch_prompt, return_log_probs=True)
    # 生成的序列长度是否都一致，通过结束符判断，序列实际长度？
    seq_len = batch_response.size(1) # 生成序列长度
    
    # 奖励模型评分
    # 拼接:提示词+SEP_TOKEN_ID+生成的回答
    batch_data = torch.cat([batch_prompt, torch.tensor([[SEP_TOKEN_ID]] * batch_size).to(batch_prompt.device), batch_response], dim=1)
    # 获取每个回答的得分(累积得分)
    batch_scores = reward_model(batch_data) # 形状: [batch_size]
    batch_scores = torch.clamp(batch_scores, -10.0, 10.0) # 奖励裁剪
    
    # 分配奖励（序列级奖励，仅最后时间步非零）
    rewards = torch.zeros(batch_size, seq_len).to(batch_scores.device)
    # 这么来看，reward_model输出的是最后一步的奖励吗？而不是序列的累积奖励值？
    rewards[:, -1] = batch_scores
    
    # Actor旧策略概率
    # old_actor_logits = actor_model.forward(batch_prompt) # [batch_size, seq_len, vocab_size]
    """ 
    很奇怪啊，这里前向传播，输出每个动作的对数概率和
    batch_response, old_log_probs = actor_model.generate(batch_prompt, return_log_probs=True)
    这里面输出有什么不一样？不都是同样的吗？输出的概率不t也是同样的吗？
    除非你actor_model分为目标和主网络模型，不然我想不通，为啥还要多此一举呢？
    """
    # old_log_probs = gather_log_probs(old_actor_logits, batch_response)
    
    # Critic值
    """ 
    这个评价模型是输出每个token的状态值V(token)，还是表达V_k(batch_promp+token_0+token_1+...+token_k)
    这是一个gpt架构的模型，还是bert的模型？感觉上是bert模型更合适。我们得看后面生成的什么，和已经生成内容，来评估这个状态的得分？
    """
    critic_values = critic_model.forward(batch_prompt, batch_response) # [batch_size,seq_len]
    
    # 参考模型概率
    ref_logits = ref_model.forward(batch_prompt)  # [batch_size, seq_len, vocab_size]
    """
    参考模型：实际上是actor_target_model对不对？稳定训练过程的？
    
    通过参考模型输出原先选择动作的对数概率，来比较KL散度，尽量让ＫＬ散度不要相差太大！
    ref_logits = ref_model.forward(batch_prompt)  # [batch_size, seq_len, vocab_size]
    """
    ref_log_probs = gather_log_probs(ref_logits, batch_response) # [batch_size, seq_len]

    # KL散度公式正确吗？怎么感觉不对呢？不是前面还应该乘上old_probs吗？
    # KL散度
    kls = (old_log_probs - ref_log_probs).mean(dim=1)  # [batch_size]
    
    # GAE优势
    """ 
    原来，只有最后一步有奖励值。前面步数的奖励值都为0
    靠状态价值模型预测输出每个动作选择的优势？
    """
    advantages = compute_advantages(critic_values, rewards, gamma, lambda_)
    
    returns = advantages + critic_values
    
    # PPO训练
    for _ in range(epochs):
        # 新策略概率
        """ 
        这里由于模型已经在做梯度更新了，所以给出的策略是一直在变的。那么代表着输出token的新概率，即新策略
        """
        new_actor_logits = actor.model.forward(batch_prompt)
        new_log_probs = gather_log_probs(new_actor_logits, batch_response)
        
        # 新值
        new_critic_values = critic_model.forward(batch_prompt, batch_response)
        
        # 概率比
        """ 
        这是类似重要性采样，将原先概率分布采样得到的token序列映射到现有概率分布中来
        """
        ratio = torch.exp(new_log_probs - old_log_probs) # [batch_size, seq_len]
        
        """ 
        我的疑问：
        这里的old_log_probs是不是with torch.no_grad()的？
        advantages　是不是也是with torch.no_grad()的？
        在计算损失时，梯度更新的只有new_log_probs
        """
        # PPO损失
        surr1 = ratio * advantages
        # 优势裁剪，防止策略过度更新
        surr2 = torch.clamp(ratio, 1 - epsilon, 1 + epsilon) * advantages
        loss_ppo = -torch.mean(torch.min(surr1, surr2))
        
        """ 
        我的疑问：
        这里的returns是不是with torch.no_grad()的？
        在计算损失时，梯度更新的只有new_critic_values
        """
        # 值损失
        loss_state_value = torch.mean((returns - new_critic_values)**2)
        
        
        """ 
        beta*kls.mean() 这一项，是kls = (old_log_probs - ref_log_probs).mean(dim=1)  # [batch_size]
        他不会进行梯度下降吧？ 他是老策略与参考模型的kls计算，不会进行梯度下降吧？只是一个正则项，对吗？
        真正进行梯度计算的是不是只有loss_ppo中的new_log_probs和value_loss_rate中的new_critic_values？
        """
        loss = loss_ppo + value_loss_rate*loss_state_value + beta*kls.mean()
        
        # 优化
        actor_optimizer.zero_grad()
        critic_optimizer.zero_grad()
        loss.backward()
        actor_optimizer.step()
        critic_optimizer.step()

NameError: name 'actor_model' is not defined

# 修正版

有诸多问题...

In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim

# 超参数
gamma = 0.99
lambda_ = 0.95
epsilon = 0.2
beta = 0.01
value_loss_rate = 0.5
epochs = 10
SEP_TOKEN_ID = 102  # 假设的分隔符Token ID
PAD_TOKEN_ID = 0  # 填充Token ID

# 一般是bert或gpt模型，这里只是示例，一般会和actor模型共享几个层
# Critic模型示例
class CriticModel(nn.Module):
    def __init__(self, bert_model, hidden_size):
        super().__init__()
        self.bert = bert_model
        self.value_head = nn.Linear(hidden_size, 1)
    
    def forward(self, prompts, responses):
        inputs = torch.cat([prompts, responses], dim=1)
        outputs = self.bert(inputs).last_hidden_state
        values = self.value_head(outputs).squeeze(-1)  # [batch_size, seq_len]
        return values
    
# 优化器
actor_optimizer = optim.Adam(actor_model.parameters(), lr=1e-5)
critic_optimizer = optim.Adam(critic_model.parameters(), lr=1e-5)

# 工具函数
def gather_log_probs(logits, actions):
    return torch.log_softmax(logits, dim=-1).gather(dim=-1, index=actions.unsqueeze(-1)).squeeze(-1)

def compute_advantages(values, rewards, gamma=0.99, lambda_=0.95):
    advantages = torch.zeros_like(rewards)
    gae = 0
    for t in reversed(range(rewards.size(1))):
        delta = rewards[:, t] + gamma * values[:, t+1] * (t < rewards.size(1)-1) - values[:, t]
        gae = delta + gamma * lambda_ * gae
        advantages[:, t] = gae
    return advantages

# 训练循环
for batch_prompt in prompt_dataset:
    batch_size, prompt_len = batch_prompt.size()  # [batch_size, prompt_len]
    prompt_mask = (batch_prompt != PAD_TOKEN_ID).long()  # 注意力掩码
    
    # 生成响应
    with torch.no_grad():
        batch_response, old_log_probs = actor_model.generate(batch_prompt, max_length=100, return_log_probs=True)
    seq_len = batch_response.size(1)
    response_mask = (batch_response != PAD_TOKEN_ID).long()
    
    # 奖励模型评分
    batch_data = torch.cat([batch_prompt, torch.tensor([[SEP_TOKEN_ID]] * batch_size).to(batch_prompt.device), batch_response], dim=1)
    batch_scores = reward_model(batch_data)  # [batch_size]
    batch_scores = torch.clamp(batch_scores, -10.0, 10.0)
    
    # 分配奖励
    rewards = torch.zeros(batch_size, seq_len).to(batch_scores.device)
    rewards[:, -1] = batch_scores
    
    # Critic值
    critic_values = critic_model(batch_prompt, batch_response)  # [batch_size, seq_len]
    
    # GAE优势和回报
    advantages = compute_advantages(critic_values.detach(), rewards, gamma, lambda_)
    returns = (advantages + critic_values).detach()
    
    # 参考模型概率
    ref_logits = ref_model.forward(batch_prompt)  # [batch_size, seq_len, vocab_size]
    ref_log_probs = gather_log_probs(ref_logits, batch_response)
    
    # PPO训练
    for _ in range(epochs):
        # 新策略概率
        new_actor_logits = actor_model.forward(batch_prompt)
        new_log_probs = gather_log_probs(new_actor_logits, batch_response)
        
        # 新值
        new_critic_values = critic_model.forward(batch_prompt, batch_response)
        
        # 概率比
        ratio = torch.exp(new_log_probs - old_log_probs.detach())
        
        # PPO损失
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1-epsilon, 1+epsilon) * advantages
        loss_ppo = -torch.mean(torch.min(surr1, surr2) * response_mask)
        
        # 值损失
        loss_state_value = torch.mean(((returns - new_critic_values)**2) * response_mask)
        
        # KL散度（基于新策略）
        new_probs = torch.exp(new_log_probs)
        ref_probs = torch.exp(ref_log_probs)
        kls = new_probs * (new_log_probs - ref_log_probs)
        kls = kls.sum(dim=1) * response_mask[:, -1]  # 考虑序列级KL
        
        # 总损失
        loss = loss_ppo + value_less_rate * loss_state_value + beta * kls.mean()
        
        # 优化
        actor_optimizer.zero_grad()
        critic_optimizer.zero_grad()
        loss.backward()
        actor_optimizer.step()
        critic_optimizer.step()